In [11]:
links = {
    'webpage-1': set(['webpage-2', 'webpage-4', 'webpage-5', 'webpage-6', 'webpage-8', 'webpage-9', 'webpage-10']),
    'webpage-2': set(['webpage-5', 'webpage-6']),
    'webpage-3': set(['webpage-10']),
    'webpage-4': set(['webpage-9']),
    'webpage-5': set(['webpage-2', 'webpage-4']),
    'webpage-6': set([]), # dangling page
    'webpage-7': set(['webpage-1', 'webpage-3', 'webpage-4']),
    'webpage-8': set(['webpage-1']),
    'webpage-9': set(['webpage-1', 'webpage-2', 'webpage-3', 'webpage-8', 'webpage-10']),
    'webpage-10': set(['webpage-2', 'webpage-3', 'webpage-8', 'webpage-9']),
}

In [12]:
def build_index(links):
    website_list = links.keys()
    return {website: index for index, website in enumerate(website_list)}
 
website_index = build_index(links)
print(website_index)
# {'webpage-10': 3, 'webpage-9': 0, 'webpage-8': 1, 'webpage-1': 2, 'webpage-3': 4, 'webpage-2': 5, 'webpage-5': 6, 'webpage-4': 7, 'webpage-7': 8, 'webpage-6': 9}


{'webpage-1': 0, 'webpage-2': 1, 'webpage-3': 2, 'webpage-4': 3, 'webpage-5': 4, 'webpage-6': 5, 'webpage-7': 6, 'webpage-8': 7, 'webpage-9': 8, 'webpage-10': 9}


In [13]:
import numpy as np
 
def build_transition_matrix(links, index):
    total_links = 0
    A = np.zeros((len(index), len(index)))
    for webpage in links:
        # dangling page
        if not links[webpage]:
            # Assign equal probabilities to transition to all the other pages
            A[index[webpage]] = np.ones(len(index)) / len(index)
        else:
            for dest_webpage in links[webpage]:
                total_links += 1
                A[index[webpage]][index[dest_webpage]] = 1.0 / len(links[webpage])
 
    return A
 
A = build_transition_matrix(links, website_index)
print(A)

[[0.         0.14285714 0.         0.14285714 0.14285714 0.14285714
  0.         0.14285714 0.14285714 0.14285714]
 [0.         0.         0.         0.         0.5        0.5
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         1.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         1.         0.        ]
 [0.         0.5        0.         0.5        0.         0.
  0.         0.         0.         0.        ]
 [0.1        0.1        0.1        0.1        0.1        0.1
  0.1        0.1        0.1        0.1       ]
 [0.33333333 0.         0.33333333 0.33333333 0.         0.
  0.         0.         0.         0.        ]
 [1.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.2        0.2        0.2        0.         0.         0.
  0.         0.2        0.         0.2       ]
 [0.         0.25       0.2

In [14]:
def pagerank(A, eps=0.0001, d=0.85):
    P = np.ones(len(A)) / len(A)
    while True:
        new_P = np.ones(len(A)) * (1 - d) / len(A) + d * A.T.dot(P)
        delta = abs(new_P - P).sum()
        if delta <= eps:
            return new_P
        P = new_P
 
results = pagerank(A)
 
print("Results:", results) # [ 0.13933698,  0.09044235,  0.1300934 ,  0.13148714,  0.08116465, 0.1305122 ,  0.09427366,  0.085402  ,  0.02301397,  0.09427366]
print(sum(results)) # 1.0
print([item[0] for item in sorted(enumerate(results), key=lambda item: -item[1])]) # [0, 3, 5, 2, 6, 9, 1, 7, 4, 8]
 

Results: [0.1300934  0.1305122  0.08116465 0.085402   0.09427366 0.09427366
 0.02301397 0.09044235 0.13933698 0.13148714]
1.0
[8, 9, 1, 0, 4, 5, 7, 3, 2, 6]


In [15]:
from nltk.corpus import brown, stopwords
from nltk.cluster.util import cosine_distance
 
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)
 
# One out of 5 words differ => 0.8 similarity
print(sentence_similarity("This is a good sentence".split(), "This is a bad sentence".split()))
 
# One out of 2 non-stop words differ => 0.5 similarity
print(sentence_similarity("This is a good sentence".split(), "This is a bad sentence".split(), stopwords.words('english')))
 
# 0 out of 2 non-stop words differ => 1 similarity (identical sentences)
print(sentence_similarity("This is a good sentence".split(), "This is a good sentence".split(), stopwords.words('english')))
 
# Completely different sentences=> 0.0
print(sentence_similarity("This is a good sentence".split(), "I want to go to the market".split(), stopwords.words('english')))
 

0.7999999999999998
0.4999999999999999
0.9999999999999998
0.0


In [16]:
import numpy as np
  
# Get a text from the Brown Corpus
sentences = brown.sents('ca01')
 
print(sentences)
# [[u'The', u'Fulton', u'County', u'Grand', u'Jury', u'said', u'Friday', u'an', u'investigation', u'of', u"Atlanta's", u'recent', u'primary', u'election', u'produced', u'``', u'no', u'evidence', u"''", u'that', u'any', u'irregularities', u'took', u'place', u'.'], [u'The', u'jury', u'further', u'said', u'in', u'term-end', u'presentments', u'that', u'the', u'City', u'Executive', u'Committee', u',', u'which', u'had', u'over-all', u'charge', u'of', u'the', u'election', u',', u'``', u'deserves', u'the', u'praise', u'and', u'thanks', u'of', u'the', u'City', u'of', u'Atlanta', u"''", u'for', u'the', u'manner', u'in', u'which', u'the', u'election', u'was', u'conducted', u'.'], ...]
 
print(len(sentences))  #  98
 
# get the english list of stopwords
stop_words = stopwords.words('english')

def build_similarity_matrix(sentences, stopwords=None):
    # Create an empty similarity matrix
    S = np.zeros((len(sentences), len(sentences)))
 
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:
                continue
 
            S[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
 
    # normalize the matrix row-wise
    for idx in range(len(S)):
        S[idx] /= S[idx].sum()
 
    return S
 
S = build_similarity_matrix(sentences, stop_words)    
print(S)

[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], ['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.'], ...]
98
[[0.         0.02171602 0.02438459 ... 0.01056602 0.02113203 0.0224139 ]
 [0.01642812 0.         0.00853223 ... 0.00646989 0.01940966 0.0137247 ]
 [0.0326456  0.01509956 0.         ... 0.00642841 0.01928522 0.02727342]
 ...
 [0.0154814  0.01253106 0.00703547 ... 0.         0.03200945 0.0150894 ]
 [0.01453939 0.01765287 0.00991107 ... 0.01503088 0.         0.02125687]
 [0.0179617  0

In [17]:
from operator import itemgetter 
 
sentence_ranks = pagerank(S)
 
print(sentence_ranks)
 
# Get the sentences ordered by rank
ranked_sentence_indexes = [item[0] for item in sorted(enumerate(sentence_ranks), key=lambda item: -item[1])]
print(ranked_sentence_indexes)
 
# Suppose we want the 5 most import sentences
SUMMARY_SIZE = 5
SELECTED_SENTENCES = sorted(ranked_sentence_indexes[:SUMMARY_SIZE])
print(SELECTED_SENTENCES)
 
# Fetch the most important sentences
summary = itemgetter(*SELECTED_SENTENCES)(sentences)
 
# Print the actual summary
for sentence in summary:
    print(' '.join(sentence))

[0.01121618 0.01416586 0.00875955 0.01720666 0.01097157 0.00992082
 0.01200115 0.00168689 0.01309704 0.01454477 0.01097302 0.00816622
 0.00878952 0.00681878 0.01487246 0.00980958 0.01631003 0.01077275
 0.01016464 0.00164935 0.01201909 0.01345858 0.01054308 0.01246479
 0.00189483 0.00848166 0.01189495 0.00226414 0.00871032 0.01280831
 0.01201529 0.0091875  0.0132896  0.01413383 0.0083716  0.01075297
 0.01131163 0.00866148 0.00781889 0.00798044 0.01358306 0.00816582
 0.00787996 0.00986716 0.01055781 0.01245516 0.00559471 0.00901723
 0.01349356 0.01029195 0.00615882 0.00646616 0.00680286 0.01227652
 0.00915185 0.01146637 0.01031809 0.00676416 0.00947115 0.00730678
 0.00687184 0.0022958  0.01043746 0.00837449 0.00795457 0.00182517
 0.0082859  0.00822529 0.01303836 0.00667884 0.00853634 0.01013017
 0.00852789 0.01224895 0.00630303 0.01017482 0.01098564 0.01494719
 0.00789689 0.01391384 0.01242932 0.0017009  0.01639154 0.01258241
 0.01145775 0.01336677 0.01761915 0.01135941 0.01403542 0.0138

In [18]:
def textrank(sentences, top_n=5, stopwords=None):
    """
    sentences = a list of sentences [[w11, w12, ...], [w21, w22, ...], ...]
    top_n = how may sentences the summary should contain
    stopwords = a list of stopwords
    """
    S = build_similarity_matrix(sentences, stop_words) 
    sentence_ranks = pagerank(S)
 
    # Sort the sentence ranks
    ranked_sentence_indexes = [item[0] for item in sorted(enumerate(sentence_ranks), key=lambda item: -item[1])]
    selected_sentences = sorted(ranked_sentence_indexes[:top_n])
    summary = itemgetter(*selected_sentences)(sentences)
    return summary
 
for idx, sentence in enumerate(textrank(sentences, stopwords=stopwords.words('english'))):
    print("%s. %s" % ((idx + 1), ' '.join(sentence)))


1. `` Only a relative handful of such reports was received '' , the jury said , `` considering the widespread interest in the election , the number of voters and the size of this city '' .
2. Nevertheless , `` we feel that in the future Fulton County should receive some portion of these available funds '' , the jurors said .
3. -- After a long , hot controversy , Miller County has a new school superintendent , elected , as a policeman put it , in the `` coolest election I ever saw in this county '' .
4. `` This was the coolest , calmest election I ever saw '' , Colquitt Policeman Tom Williams said .
5. `` Everything went real smooth '' , the sheriff said .


# Testing

In [19]:
sentences = """Architecturally, the school has a Catholic character.
Atop the Main Building's gold dome is a golden statue of the Virgin Mary.
Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes".
Next to the Main Building is the Basilica of the Sacred Heart.
Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection.
It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858.
At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary."""

In [22]:
text = """The koala (Phascolarctos cinereus, or, inaccurately, koala bear[a]) is an arboreal herbivorous marsupial native to Australia. It is the only extant representative of the family Phascolarctidae and its closest living relatives are the wombats, which comprise the family Vombatidae. The koala is found in coastal areas of the mainland's eastern and southern regions, inhabiting Queensland, New South Wales, Victoria, and South Australia. It is easily recognisable by its stout, tailless body and large head with round, fluffy ears and large, spoon-shaped nose. The koala has a body length of 60–85 cm (24–33 in) and weighs 4–15 kg (9–33 lb). Fur colour ranges from silver grey to chocolate brown. Koalas from the northern populations are typically smaller and lighter in colour than their counterparts further south. These populations possibly are separate subspecies, but this is disputed.

Koalas typically inhabit open eucalypt woodlands, and the leaves of these trees make up most of their diet. Because this eucalypt diet has limited nutritional and caloric content, koalas are largely sedentary and sleep up to 20 hours a day. They are asocial animals, and bonding exists only between mothers and dependent offspring. Adult males communicate with loud bellows that intimidate rivals and attract mates. Males mark their presence with secretions from scent glands located on their chests. Being marsupials, koalas give birth to underdeveloped young that crawl into their mothers' pouches, where they stay for the first six to seven months of their lives. These young koalas, known as joeys, are fully weaned around a year old. Koalas have few natural predators and parasites, but are threatened by various pathogens, such as Chlamydiaceae bacteria and the koala retrovirus.

Koalas were hunted by Indigenous Australians and depicted in myths and cave art for millennia. The first recorded encounter between a European and a koala was in 1798, and an image of the animal was published in 1810 by naturalist George Perry. Botanist Robert Brown wrote the first detailed scientific description of the koala in 1814, although his work remained unpublished for 180 years. Popular artist John Gould illustrated and described the koala, introducing the species to the general British public. Further details about the animal's biology were revealed in the 19th century by several English scientists. Because of its distinctive appearance, the koala is recognised worldwide as a symbol of Australia. Koalas are listed as Vulnerable by the International Union for Conservation of Nature. The animal was hunted heavily in the early 20th century for its fur, and large-scale cullings in Queensland resulted in a public outcry that initiated a movement to protect the species. Sanctuaries were established, and translocation efforts moved to new regions koalas whose habitat had become fragmented or reduced. Among the many threats to their existence are habitat destruction caused by agriculture, urbanisation, droughts and associated bushfires, some related to climate change.[4] Increased habitat loss may also increase risks from vehicle traffic, dog attacks, pesticides in waterways, and increased food competition.[5][6][7]"""

In [24]:
textrank(text, 5, stopwords=stopwords.words('english'))

KeyboardInterrupt: 